In [24]:
#import of packages
import networkx as nx
import matplotlib.pyplot as plt
import random  
import numpy as np
import sys
from  fractions import Fraction
import timeit
from time import sleep
from collections import defaultdict
import heapq as heap
from tqdm import tqdm
from community import community_louvain
import collections 
import matplotlib.cm as cm
import statistics

from IPython.display import HTML, display
from itertools import chain, combinations
from itertools import islice
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from joblib import Parallel, delayed
import csv
from pprint import pprint
import math
from operator import itemgetter
import itertools
import scipy
from scipy import stats
from IPython.core.magic import register_line_magic
from IPython.display import HTML, display
import json
import powerlaw
from decimal import Decimal

In [25]:
def get_communities(part,G): # returns the number of communities in G given the partition part
    communities = []
    for i in G.nodes():
        if part[i] not in communities:
            communities.append(part[i])
    return len(communities)

In [35]:
def path_blocked_MIA(from_node,to_node, MIIA, S): # if a negative information needs to go through a node in S_P the path is blocked
    start=timeit.default_timer()
    current_node=from_node
    if current_node in S:
        return True
    while current_node != to_node and list(MIIA.successors(current_node)):
        if list(MIIA.successors(current_node))[0] in S:
            return True
        else:
            current_node=list(MIIA.successors(current_node))[0]
    return False

def dijkstra_MIA(startingNode, G_path, G_deg,in_edges,min_prob,S_N,p_n): # dijkstra's algorithm to compute the MIIA
    
    MIA = nx.DiGraph()
    MIA.add_node(startingNode)

    prob,probabilities={startingNode: 1},{startingNode:1}
    steps,steps_final={startingNode:0},{startingNode:0}
    parentsMap={}
    
    pq = []
    nodeCosts = defaultdict(lambda: float('inf'))
    nodeCosts[startingNode] = 0
    heap.heappush(pq, (0, startingNode))
 
    while pq:
        # go greedily by always extending the shorter cost nodes first
        _, node = heap.heappop(pq)
        if node in S_N: continue
        if not in_edges:
            neighbors =list(G_path.successors(node))
        else:
            neighbors = list(G_path.predecessors(node))
        for adjNode in neighbors:  
            if not in_edges:    
                weight= -np.log(G_deg[node][adjNode][p_n])
                prob_val= round(G_deg[node][adjNode][p_n]*prob[node],3)
            else:
                weight= -np.log(G_deg[adjNode][node][p_n])
                prob_val= round(G_deg[adjNode][node][p_n]*prob[node],3)
            newCost = nodeCosts[node] + weight
            if nodeCosts[adjNode] > newCost and prob_val>=min_prob :
                parentsMap[adjNode]=node
                nodeCosts[adjNode] = newCost
                prob[adjNode]= prob_val
                steps[adjNode]=steps[node]+1
                heap.heappush(pq, (newCost, adjNode))
                
    for w in  parentsMap.keys() : 
        if  in_edges:
            MIA.add_edge(w,parentsMap[w])
        else:
            MIA.add_edge(parentsMap[w],w)
        probabilities[w]= prob[w]
        steps_final[w]= steps[w]
        
    return  MIA,steps_final,probabilities # we return the MIA and the parentMap in case we need to reconstruct the MIA

In [36]:
def get_p_value(v,pos,t,S_N,S_P):# initializes the values of P_p_t, P_n_t, ap_n_t and ap_p_t in compute_ap_N
    if v in S_P:
        if pos:
            if t==0: return 1,1 
            else: return 0,1
        else: return 0,0
            
    elif v in S_N:
        if pos: return 0,0
        else: 
            if t==0: return 1,1 
            else: 0,1
    else: return 0,0 

def compute_ap_N(u,S_N,S_P,MIA_n,MIA_p,G): #probability of transmission is inversly proportional to the in-degree  
    
    POS_t=[v for v in S_P if v in MIA_p.nodes() ]
    NEG_t=[v for v in S_N if v in MIA_n.nodes()]
    POS_next=set()
    NEG_next=set()
    
    P_p_t={}
    P_p_next={}
    P_n_t={}
    P_n_next={}
    
    ap_n_t={}
    ap_p_t={}
    for node in list(MIA_n.nodes())+list(MIA_p.nodes()):
         
        P_p_t[node],ap_p_t[node] = get_p_value(node,True,0,S_N,S_P)
        P_n_t[node],ap_n_t[node] = get_p_value(node,False,0,S_N,S_P)
    t=0
    
    while NEG_t:
        
        ap_n_next=ap_n_t.copy()
        ap_p_next=ap_p_t.copy()
        
        
        temp_P={k:1 for k in list(MIA_n.nodes())+list(MIA_p.nodes())}
        temp_N={k:1 for k in list(MIA_n.nodes())+list(MIA_p.nodes())}
        
        for v in POS_t: # positive influence has priority
            
            if not list(MIA_p.successors(v)):
                continue
            for w in list(MIA_p.successors(v)):
                POS_next.add(w)
                temp_P[w]*=(1-P_p_t[v]*G[v][w]['positive'])
            
        for v in POS_next:
            
            P_p_next[v]= (1-temp_P[v])*(1-ap_n_t[v])*(1-ap_p_t[v])
            ap_p_next[v]=ap_p_t[v]+P_p_next[v]
             
        for v in NEG_t :
            if not list(MIA_n.successors(v)):
                continue 
            for w in list(MIA_n.successors(v)):
                NEG_next.add(w)
                temp_N[w]*=(1-P_n_t[v]*G[v][w]['negative'])
            
        for v in NEG_next:
            P_n_next[v] = temp_P[v]*(1-temp_N[v])*(1-ap_n_t[v])*(1-ap_p_t[v])
            ap_n_next[v]=ap_n_t[v]+P_n_next[v] 
            
        if list(POS_next): 
            ap_p_t = ap_p_next.copy()  
            
        if list(NEG_next):
            
            ap_n_t = ap_n_next.copy() 
             
        P_p_t,P_n_t,NEG_t,POS_t = P_p_next,P_n_next,NEG_next,POS_next
        POS_next=set()
        NEG_next=set()
        t+=1
    
    return ap_n_t[u]



In [37]:
# this version of CMIA-O tries to help the community where the least percentage of nodes has been saved
def CMIA_O_fair_new(G,S_N,k,min_prob,partition): # we now consider the communities, and take as input the partition
    avg_deg=sum([G.out_degree(x) for x in G.nodes()])/len(G.nodes())
    S_P=[]
    Neg_range=set()# nodes that could be reached by S_N with probability higer than min_prob
    communities=get_communities(partition,G)
    MIA_edges_n,MIA_p_edges={},{}# to keep track of the MIIAs without having to recompute them
    DecInf_per_comm={}# dictionary storing the contribution per community a node has if added to S_P 
    E_inf_num=[0]*communities # value of expected infected nodes per community
    saved_per_comm=[0]*communities # expected number of saved nodes per community
    steps_n,steps_p={},{}
    proba_n,proba_p={},{}
    
    inf=[]
    for u in S_N:
        MIOA,s,p=dijkstra_MIA(u, G,G, False, min_prob,[],"negative")
        for v in list(set(MIOA.nodes()) - set(S_N)- set(Neg_range)):
            Neg_range.add(v) # contains the nodes that could get infected
            inf.append((v,u))
    print(len(Neg_range))
    for u in  Neg_range:# for each node that gets ifected  we compute how much the probability of infection value lowers if we add a node to the seed
        
        MIIA,steps,probs =dijkstra_MIA(u, G,G, True, min_prob,S_N,"negative") # our candidates are the nodes that reach u with probability higher than min_prob
        MIA_edges_n[u]= [e for e in MIIA.edges()].copy() #skeleton of MIIA
        steps_n[u]= {key:val for key, val in steps.items()}
        
        
            
        MIIA_p,steps_pos,probs_p = dijkstra_MIA(u, G,G, True, min_prob,S_N,"positive") # our candidates are the nodes that reach u with probability higher than min_prob
        MIA_p_edges[u]= [e for e in MIIA_p.edges()].copy() #skeleton of MIIA
        steps_p[u]= {key:val for key, val in steps_pos.items()}
        
        ap=compute_ap_N(u,S_N, S_P, MIIA, MIIA_p, G)# probability of infection of u if we dont intervene
        if ap==0: 
            continue
        
        
        infectees=[x for x in MIIA.nodes() if x in S_N]
        
        longest= [x for _, x in sorted(zip([steps[x] for x in infectees], infectees), reverse=True)][0]
        
        E_inf_num[partition[u]]+=ap # we add the propability of infection of u to our expected value within its community
        
        for v in [x for x in list(set(MIIA_p.nodes())-set(S_N)) if steps_pos[x]<= steps[longest]] :
            if len(infectees)==1 and path_blocked_MIA(u,infectees, MIIA, [v]): ap_v=0
            else: ap_v= compute_ap_N(u,S_N, S_P + [v], MIIA,MIIA_p ,G) # probability of infection of u if v is added to S_P
            
            if v in DecInf_per_comm:
                DecInf_per_comm[v][partition[u]]+= (ap-ap_v)
            else:
                DecInf_per_comm[v]=[0]*communities
                DecInf_per_comm[v][partition[u]]= (ap-ap_v) #compute total impact of the node as truth campaigner per community 
           
    E_inf_num=[round(x,1) for x in E_inf_num] 
    
    while len(S_P)< k:
        
        percentage_saved=[float(saved_per_comm[c]/E_inf_num[c]) if E_inf_num[c]!=0 else 1  for c in range(communities) ]
        need_saving=[c for c in range(communities) if percentage_saved[c]==min(percentage_saved)]# communities with minimum percentage of saved nodes
        DecInf_needed={v:sum([DecInf_per_comm[v][c] for c in need_saving ]) for v in DecInf_per_comm} # contibution of each node within the communities that need_saving
            
        Saved_per_com_v={} # dictionary storing percentage of nodes we expect to save if we add v to S_P per community
        for key in DecInf_per_comm.keys():
            Saved_per_com_v[key]=[float((saved_per_comm[c]+DecInf_per_comm[key][c])/E_inf_num[c]) if E_inf_num[c]!=0 else 1 for c in range(communities)]
                
        candidates={n:min(v) for n,v in Saved_per_com_v.items() if n not in S_N+list(S_P) and len(S_P+[n])<= k} # we store for each node the min percentage of saved nodes in a community if added to S_P
        candidates=dict(sorted(candidates.items(), key=lambda kv: DecInf_needed[kv[0]], reverse=True)) # some might tie, so we order them in descending order of DecInf_needed vaue
        if len(candidates)==0:break    
         
        u = max(candidates,key=candidates.get) # chose node maximin value that has the biggest contribution within the communities with min saved percentage
        
        saved_per_comm=[DecInf_per_comm[u][c]+saved_per_comm[c] for c in range(communities)] # we update the expected number of saved nodes
        #we now proceed to update the contribution of each node
        MIOA,_,_=dijkstra_MIA(u, G,G, False, min_prob, S_N+S_P,"positive")
        
        for v in [x for x in MIOA.nodes() if x in Neg_range]: # nodes that u could potentially save  
            
            MIIA.clear() # we recontruct the MIIA, which we already computed and stored
           
            MIIA.add_edges_from(MIA_edges_n[v]) 
           
            
            MIIA_p.clear() # we recontruct the MIIA, which we already computed and stored
            MIIA_p.add_edges_from(MIA_p_edges[v]) 
            
           
            
            longest= max([x for x in steps_n[v].items() if x[0] in S_N and not path_blocked_MIA(x[0],v, MIIA, S_P)],key=lambda item:item[1], default=(None,None))[0]
            if longest==None:continue
            
            ap=compute_ap_N(v,S_N,S_P,MIIA,MIIA_p ,G) #probability of infection of v with current S_P 
            ap_u=compute_ap_N(v,S_N,S_P+[u],MIIA,MIIA_p,G) #probability of infection of v with current S_P which now includes u 
            for w in [x for x in list(set(MIIA_p.nodes())-set(S_N)) if steps_p[v][x]<= steps_n[v][longest] ]: #nodes whose contribution now decreses since u helps 
                #if  path_blocked_MIA(w,v, MIIA, S_N):continue  # if we have to go through a negative seed to reach v, we cant save him with w and we continue
                ap_w=compute_ap_N(v,S_N, S_P + [w], MIIA,MIIA_p ,G) # probability of infection of v if w is added to S_P
                
                DecInf_per_comm[w][partition[v]]-=(ap-ap_w) #remove all influence of w when u was not in S_P  
                if ap<min_prob: continue
                if v!=u or ap_u==0:    
                    ap_u_w=compute_ap_N(v,S_N,S_P +[w]+[u],MIIA,MIIA_p ,G)
                    DecInf_per_comm[w][partition[v]]+=(ap_u-ap_u_w)
                    
               
        S_P.append(u)
        
    saved_per_comm_final=[float((saved_per_comm[c]+DecInf_per_comm[key][c])/E_inf_num[c]) if E_inf_num[c]!=0 else 1 for c in range(communities)]     
              
    return S_P 

In [38]:
#code to set probability of the edges
def set_weight_degree(G,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= 1/G.in_degree(v)
    return G

def set_weight_trivalency(G,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= random.choice([0.1,0.01,0.001])
    return G

def set_weight_value(G,val,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= round(val,3)
    return G

def set_weight_average(G,p_n):
    avg=sum([G.out_degree(x) for x in G.nodes()])/len(G.nodes())
    for u,v in G.edges():
        G[u][v][p_n]= 1/avg
    return G

def set_weight_comm(G,inter,intra,p_n):
    for u,v in G.edges():
        if G.nodes[u]['community']== G.nodes[v]['community']:
            G[u][v][p_n]= inter
        else:
            G[u][v][p_n]= intra
    return G

In [39]:
network=nx.erdos_renyi_graph(1000,0.01,directed=True)


# the network nodes need to have a lavel commuity defining to which community they belong to
partition=community_louvain.best_partition(network.to_undirected())
for node in network.nodes():
    network.nodes[node]['community']=partition[node]




# the network shall also have allocated to each edge the probability of infection        
network=set_weight_degree(network,"positive")
network=set_weight_degree(network,"negative")

k_p=100
k_n=50
S_N=random.choices([v for v in list(network.nodes())],k= k_n)
S_P=CMIA_O_fair_new(network,S_N,k_p,0.01,partition)
print(S_P)

873


100%|████████████████████████████████████████████████████████████████████████████████| 873/873 [00:19<00:00, 44.17it/s]


[415, 853, 685, 782, 320, 403, 701, 819, 1, 190, 802, 24, 268, 446, 113, 379, 988, 208, 836, 5, 767, 887, 302, 345, 530, 902, 204, 613, 25, 864, 250, 749, 826, 82, 713, 137, 158, 844, 271, 838, 719, 827, 854, 912, 776, 804, 732, 77, 103, 594, 441, 842, 957, 6, 584, 257, 295, 742, 331, 848, 290, 313, 748, 636, 511, 820, 235, 897, 2, 279, 366, 920, 996, 424, 377, 807, 892, 510, 336, 600, 856, 973, 462, 855, 745, 389, 40, 349, 189, 908, 806, 861, 529, 730, 227, 225, 495, 587, 669, 828]
